In [ ]:
import tensorflow as tf
import cv2 as cv
import numpy as np

In [ ]:
%%capture
!pip install gradio

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## **Find Tea Leaves from Other Leaves**

**Load Model_**

In [ ]:
from keras.models import model_from_json

# Load the model architecture from JSON
with open("/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_01/model_02.json", "r") as json_file:
    loaded_model_02_json = json_file.read()
loaded_model_02 = model_from_json(loaded_model_02_json)

# Load the model weights
loaded_model_02.load_weights("/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_01/model_02_weights.h5")

print("Model 02 loaded from disk.")

Model 02 loaded from disk.


**Define Prediction classes**

In [ ]:
class_labels = ['Alstonia Scholaris (P2)-diseased',
 'Alstonia Scholaris (P2)-healthy',
 'Arjun (P1)-diseased',
 'Arjun (P1)-healthy',
 'Bael (P4)-diseased',
 'Basil (P8)-healthy',
 'Chinar (P11)-diseased',
 'Chinar (P11)-healthy',
 'Gauva (P3)-diseased',
 'Gauva (P3)-healthy',
 'Jamun (P5)-diseased',
 'Jamun (P5)-healthy',
 'Jatropha (P6)-diseased',
 'Jatropha (P6)-healthy',
 'Lemon (P10)-diseased',
 'Lemon (P10)-healthy',
 'Mango (P0)-diseased',
 'Mango (P0)-healthy',
 'Pomegranate (P9)-diseased',
 'Pomegranate (P9)-healthy',
 'Pongamia Pinnata (P7)-diseased',
 'Pongamia Pinnata (P7)-healthy',
 'Tea (P12)-diseased',
 'Tea (P12)-healthy']

**Prediction function**

In [ ]:
import cv2
import numpy as np


def predict_leaf_model(image_path):
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (300, 200))
    input_image = np.expand_dims(resized_image, axis=0)

    predictions = loaded_model_02.predict(input_image)
    predicted_class_index = np.argmax(predictions[0])

    predicted_leaf = class_labels[predicted_class_index]
    return predicted_leaf

In [ ]:
predict_leaf_model('/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/data/New_Plant_Data/Tea (P12)/healthy/0023_0001.png')

1/1 [==============================] - 0s 221ms/step


'Tea (P12)-healthy'

Interface one

## **Tea Leave Disease Identification**

### **Approach 1**

In [ ]:
segmentation_weights = '/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_02/segmentation.h5'
classification_weights = '/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_02/classification.h5'

input_shape = (256, 256, 3)
output_shape_orig = (256, 256, 3)
output_shape_new = (65536, 3)

background_color = 0
leaf_color = 255
spot_color = 128

disease_dict = {
                'algal leaf': 0,
                'brown blight': 1,
                'greyblight': 2,
                'red leaf spot': 3,
                'white spot': 4
                }

disease_color_dict = {v : k for k, v in disease_dict.items()}

In [ ]:
segmentation_model = tf.keras.models.load_model(segmentation_weights)
classification_model = tf.keras.models.load_model(classification_weights)

In [ ]:
def preprocess_input(image_path):
    img = cv.imread(image_path)
    original_img = img

    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = np.asarray(img).astype("f")
    img = cv.resize(img, (input_shape[0], input_shape[1]), interpolation = cv.INTER_AREA)
    img_norm = img / 255.0
    return img_norm, original_img

def inference_tea_leaves(image_path):
    image_path = image_path.replace("\\", "/")
    img, original_img = preprocess_input(image_path)
    Prediction = segmentation_model.predict(np.expand_dims(img, axis=0), verbose=0)
    class_label = classification_model.predict(np.expand_dims(img, axis=0), verbose=0).argmax(axis=-1)[0]
    real_label = image_path.split("/")[-2]
    Prediction = Prediction.squeeze().argmax(axis=-1)
    Prediction = Prediction.reshape(output_shape_orig[0], output_shape_orig[1], 1)
    Prediction[Prediction == 0] = background_color
    Prediction[Prediction == 1] = leaf_color
    Prediction[Prediction == 2] = spot_color

    width_orig, height_orig = original_img.shape[1], original_img.shape[0]
    Prediction = cv.resize(Prediction, (width_orig, height_orig), interpolation = cv.INTER_NEAREST)

    return disease_color_dict[class_label]

In [ ]:
inference_tea_leaves('/content/gdrive/MyDrive/Research/sachini/tea-leave-disease-pipeline/data/processed/images/algal leaf/UNADJUSTEDNONRAW_thumb_1.jpg')

'algal leaf'


### **Approach 2**

In [ ]:
filepath = '/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_02/tea-disease-classifier.hdf5'
custom_cnn_model = tf.keras.models.load_model(filepath)

In [ ]:
def predict(test_img_path):
    test_image = tf.keras.preprocessing.image.load_img(test_img_path, target_size = (256,256)) # load image

    test_image = tf.keras.preprocessing.image.img_to_array(test_image)#/255 # convert image to np array and normalize
    test_image = np.expand_dims(test_image, axis = 0)# change dimention 3D to 4D

    result = custom_cnn_model.predict(test_image) # predict diseased palnt or not

    pred = np.argmax(result, axis=1)
    pred = pred[0]

    class_names = ['algal leaf', 'brown blight', 'greyblight', 'red leaf spot', 'white spot']

    return class_names[pred]

In [ ]:
predict('/content/gdrive/MyDrive/Research/sachini/tea-leave-disease-pipeline/data/processed/images/algal leaf/UNADJUSTEDNONRAW_thumb_1.jpg')

1/1 [==============================] - 0s 203ms/step


'brown blight'

**Load reports**

In [ ]:
import json

with open("/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_02/custom_cnn_report", "r") as fp:
  custom_cnn_report = json.load(fp)

with open("/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/Module_02/xception_report", "r") as fp:
  xception_report = json.load(fp)

In [ ]:
def gr_func(img_path):
  global disease
  # Approach 01 result
  app01_pred = inference_tea_leaves(img_path)

  # Approach 02 result
  app02_pred = predict(img_path)

  # Assign weights to F1 scores and accuracies
  f1_weight = 0.6
  accuracy_weight = 0.4

  # Calculate weighted averages
  app01_weighted_avg = (f1_weight * xception_report['weighted avg']['f1-score']) + (accuracy_weight * xception_report['accuracy'])
  app02_weighted_avg = (f1_weight * custom_cnn_report['weighted avg']['f1-score']) + (accuracy_weight * custom_cnn_report['accuracy'])

  # Compare weighted averages
  if app01_weighted_avg > app02_weighted_avg:
      best_pred = app01_pred
  elif (app02_weighted_avg > app01_weighted_avg):
      best_pred = app02_pred
  else:
      # Handle tiebreaker
      if xception_report['weighted avg']['f1-score'] > custom_cnn_report['weighted avg']['f1-score']:
          best_pred = app01_pred
      else:
          best_pred = app02_pred

  disease = best_pred

  return [app01_pred, app02_pred, best_pred]

## **Medication and Prevention Suggestion System**

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
import glob

symptoms_txt = ''
knowledge_txt = ''
medication_txt= ''

symptoms_dir_path = r'/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Disease Identification/*.*'
knowledge_dir_path = r'/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/*.*'
medication_dir_path = r'/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Medications & Preventions/*.*'

for file in glob.glob(symptoms_dir_path, recursive=True):
    print(file)
    with pdfplumber.open(file) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            symptoms_txt = symptoms_txt + '\n' + single_page_text
    pdf.close()

for file in glob.glob(knowledge_dir_path, recursive=True):
    print(file)
    with pdfplumber.open(file) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            knowledge_txt = knowledge_txt + '\n' + single_page_text
    pdf.close()

for file in glob.glob(medication_dir_path, recursive=True):
    print(file)
    with pdfplumber.open(file) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            medication_txt = medication_txt + '\n' + single_page_text
    pdf.close()

/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Disease Identification/Tea Leaf Disease and Symptoms.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/Tea.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Tea Knowledge Base/Tea - Wikipedia.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Medications & Preventions/Tea Leaf Disease Medications2.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Medications & Preventions/Tea Leaf Disease Medications3.pdf
/content/gdrive/MyDrive/Colab Notebooks/research/data/Tea/all-about-tea/Medications & Preventions/Tea Leaf Disease Medications.pdf


In [ ]:
import nltk
nltk.download('punkt')
tokens = nltk.sent_tokenize(medication_txt)
for t in tokens:
    print(t, "\n")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Tea Leaf Rust:
Description for Tea Leaf Rust: Tea Leaf Rust is a fungal disease caused by the pathogen Hemileia
vastatrix, It appears as orange to rust-colored powdery spores on the undersides of tea leaves, leading
to yellowing and defoliation. 

Medication for Tea Leaf Rust: Fungicides like copper-based sprays (copper oxychloride) or systemic
fungicides (propiconazole) can be used to control Tea Leaf Rust, These should be applied according to
the manufacturer's instructions and in consultation with agricultural experts. 

Prevention for Tea Leaf Rust: To prevent Tea Leaf Rust, it is important to maintain good sanitation
practices in tea gardens, such as removing fallen leaves and plant debris, Regular monitoring and early
detection of rust-infected plants can help in containing the spread, Additionally, planting resistant tea
cultivars can provide some level of protection. 

Tea Blister Blight:
Description for Tea Blister Blight: Tea Blister Blight is a fungal disease caused by Exob

In [ ]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [ ]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)

['tea leaf rust description tea leaf rust tea leaf rust fungal disease caused pathogen hemileia vastatrix appears orange rustcolored powdery spores undersides tea leaves leading yellowing defoliation', 'medication tea leaf rust fungicides like copperbased sprays copper oxychloride systemic fungicides propiconazole control tea leaf rust applied according manufacturers instructions consultation agricultural experts', 'prevention tea leaf rust prevent tea leaf rust important maintain good sanitation practices tea gardens removing fallen leaves plant debris regular monitoring early detection rustinfected plants help containing spread additionally planting resistant tea cultivars provide level protection', 'tea blister blight description tea blister blight tea blister blight fungal disease caused exobasidium vexans characterized raised blisterlike lesions tea leaves turn white pinkishgray disease progresses', 'medication tea blister blight fungicides containing active ingredients like manco

In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    print(index, sim, sentences[index])
    if sim > max_sim:
      max_sim = sim
      index_sim = index

  return index_sim

In [ ]:
from gensim.models import KeyedVectors
import gensim.downloader as api
from gensim import models
import numpy

try:
  v2w_model = models.KeyedVectors.load_word2vec_format(r'/content/gdrive/MyDrive/Colab Notebooks/research/data/GoogleNews-vectors-negative300.bin.gz', binary=True)
  print("w2v Model Successfully loaded")
except:
  v2w_model = api.load('word2vec-google-news-300')
  v2w_model.save(r"/content/gdrive/MyDrive/Colab Notebooks/research/data/w2vecmodel.mod")
  print("w2v Model Saved")

w2v Model Successfully loaded


In [ ]:
def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1)

In [ ]:
sent_embeddings = []
for sent in cleaned_sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

In [ ]:
def retreiveAnswer(question):
  question_embedding = getPhraseEmbedding(question, v2w_model)
  index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)
  answer = cleaned_sentences_with_stopwords[index]
  return answer

In [ ]:
question = "Prevention for for Tea Leaf Rust?"

In [ ]:
answer = retreiveAnswer(question)

0 0.3793099561233011 tea leaf rust
description for tea leaf rust tea leaf rust is a fungal disease caused by the pathogen hemileia
vastatrix it appears as orange to rustcolored powdery spores on the undersides of tea leaves leading
to yellowing and defoliation
1 0.3726532838982494 medication for tea leaf rust fungicides like copperbased sprays copper oxychloride or systemic
fungicides propiconazole can be used to control tea leaf rust these should be applied according to
the manufacturers instructions and in consultation with agricultural experts
2 0.48989256261312003 prevention for tea leaf rust to prevent tea leaf rust it is important to maintain good sanitation
practices in tea gardens such as removing fallen leaves and plant debris regular monitoring and early
detection of rustinfected plants can help in containing the spread additionally planting resistant tea
cultivars can provide some level of protection
3 0.3457864450063201 tea blister blight
description for tea blister blight 

In [ ]:
print("Question: ", question)
print("Answer: ", answer)

Question:  Prevention for for Tea Leaf Rust?
Answer:  prevention for tea leaf rust to prevent tea leaf rust it is important to maintain good sanitation
practices in tea gardens such as removing fallen leaves and plant debris regular monitoring and early
detection of rustinfected plants can help in containing the spread additionally planting resistant tea
cultivars can provide some level of protection


### Interface

In [ ]:
import gradio as gr

In [ ]:
import numpy as np
import gradio as gr

try:
    disease
except NameError:
    disease = 'algal leaf'

def clear():
  return None

def retreiveAnswer(question):
  question_embedding = getPhraseEmbedding(question, v2w_model)
  index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)
  answer = cleaned_sentences_with_stopwords[index]
  return answer

def description(disease):
    return "Description for " + disease

def medication(disease):
    return "Medication for " + disease

def prevention(disease):
    return "Prevention for " + disease

def predict_leaf_interface(image):
    image = image.reshape((image.shape[0], image.shape[1], -1))
    image_path = "temp.jpg"
    cv2.imwrite(image_path, image)
    predicted_leaf = predict_leaf_model(image_path)
    return predicted_leaf

with gr.Blocks() as demo:
    gr.Markdown("# TeaSenseAI: Integrated Intelligent System for Tea Leaf Health Assessment, Disease Identification, and Domain Expertise")
    with gr.Tab("Leaves Prediction"):
       with gr.Row():
         with gr.Column():
          image_input = gr.inputs.Image()
         with gr.Column():
          output_text = gr.outputs.Textbox()
       with gr.Row():
        tea_button = gr.Button("Find Leaf")
    with gr.Tab("Disease Prediction"):
       with gr.Row():
         with gr.Column():
          inputs = gr.Image(label='Upload the Image of the Disease Tea leaf', type='filepath')
         with gr.Column():
          outputs = [
              gr.outputs.Textbox(label="Predicted Disease - Approach 01"),
              gr.outputs.Textbox(label="Predicted Disease - Approach 02"),
              gr.outputs.Textbox(label="Optimized Ensemble Prediction")
          ]

       with gr.Row():
        disease_button = gr.Button("Find Disease")
    with gr.Tab("Medication and Prevention Suggestion"):
      gr.Markdown(
        """
     ### TeaLeafHealthAdvisor : A Smart Medication and Prevention Suggestion System for Optimal Tea Leaf Wellness
        """)
      with gr.Row():
        des_input = gr.Textbox(value=description(disease), visible=False)
        med_input = gr.Textbox(value=medication(disease), visible=False)
        prev_input = gr.Textbox(value=prevention(disease), visible=False)

        with gr.Tab("Description"):
          des_output = gr.Textbox(label="Description")
          des_btn = gr.Button("Description")
          des_btn.click(fn=retreiveAnswer, inputs=des_input, outputs=des_output)
          clear_btn = gr.Button("Clear", variant="primary")
          clear_btn.click(fn=clear, inputs=None, outputs=[des_output])

        with gr.Tab("Medications"):
          med_output = gr.Textbox(label="Medications")
          med_btn = gr.Button("Medications")
          med_btn.click(fn=retreiveAnswer, inputs=med_input, outputs=med_output)
          clear_btn = gr.Button("Clear", variant="primary")
          clear_btn.click(fn=clear, inputs=None, outputs=[med_output])

        with gr.Tab("Preventions"):
          prev_output = gr.Textbox(label="Preventions")
          prev_btn = gr.Button("Preventions")
          prev_btn.click(fn=retreiveAnswer, inputs=prev_input, outputs=prev_output)
          clear_btn = gr.Button("Clear", variant="primary")
          clear_btn.click(fn=clear, inputs=None, outputs=[prev_output])

    tea_button.click(fn=predict_leaf_interface, inputs=image_input, outputs=output_text)
    disease_button.click(gr_func, inputs=inputs,outputs=outputs)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:259: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:262: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f69b36bca3fdeae545.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
disease

'algal leaf'

## **Intelligent Disease Identification and Tea Domain Knowledge System**

In [ ]:
%%capture
!pip install transformers

In [ ]:
import torch
from transformers import BertForQuestionAnswering


bert_model = BertForQuestionAnswering.from_pretrained(r'gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering')


from transformers import BertTokenizer

bert_tokenizer = BertTokenizer.from_pretrained(r'gdrive/My Drive/Colab Notebooks/research/data/bertforquestionanswering')

bert_model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), ep

In [ ]:
def answer_question(question, answer_text):

    input_ids = bert_tokenizer.encode(question, answer_text, max_length=512, truncation=True)

    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    sep_index = input_ids.index(bert_tokenizer.sep_token_id)

    num_seg_a = sep_index + 1

    num_seg_b = len(input_ids) - num_seg_a

    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    assert len(segment_ids) == len(input_ids)

    start_scores, end_scores = bert_model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]), return_dict=False)

    all_tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)

    print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
    print(f'score: {torch.max(start_scores)}')
    score = float(torch.max(start_scores))
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    tokens = bert_tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    for i in range(answer_start + 1, answer_end + 1):

        if tokens[i][0:2] == ' ':
            answer += tokens[i][2:]

        else:
            answer += ' ' + tokens[i]
    return answer, score

In [ ]:
def ans_genarator_disease(question):
  context = symptoms_txt
  ans, score = answer_question(question,context)
  return ans

examples = [
    ["I saw a concerning disease on my farm that affects the upper leaf surface of the tea plants. It begins with the appearance of small, yellowish spots, which unfortunately gradually grow in size and transform into a distinct rusty brown color. What is the disease?"],
    ["I observe certain symptoms on my tea leaves. The affected tea leaves display small, round lesions that appear either yellow or brown in color. These lesions have a distinct shot-hole appearance, giving the leaves a unique pattern. Over time, these lesions can cause the affected leaves to undergo discoloration and distortion. It is important for me, as a farmer, to closely monitor these symptoms and take appropriate measures to address the issue in order to maintain the health and productivity of my tea plants. What is the disease?"],
    ["There are infected tea leaves show small, circular, whitish spots on the upper leaf surface. These spots may have a yellow halo and gradually enlarge, leading to blighting. What is the disease?"],
    ["there are green, brown or orange cushion-like blotches on the leaf surface, what is the disease?"]
]

demo = gr.Interface(
    fn=ans_genarator_disease,
    inputs=gr.inputs.Textbox(lines=5, label="Describe your disease (Symptoms)"),
    outputs=gr.outputs.Textbox(label="Disease"),
    examples=examples,
    title='TeaHealthAI : Intelligent Disease Identification',
)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5f30cad80682996f64.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def ans_genarator_knowledge(question):
  context = knowledge_txt
  ans, score = answer_question(question,context)
  return ans

examples = [
    ["What is Tea?"],
    ["What are the Health Benefits of Tea?"],
    ["Is tea healthier than coffee?"],
]

demo = gr.Interface(
    fn=ans_genarator_knowledge,
    inputs=gr.inputs.Textbox(lines=5, label="Ask anything about Tea"),
    outputs=gr.outputs.Textbox(label="Generated Text"),
    examples=examples,
    title='TeaHealthAI : Tea Domain Knowledge System',
)

demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(
/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c476f3649bd0bd09e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
